This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [6]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

FsdycwexurRKxfsMkmuk


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [79]:
# First, import the relevant modules
import requests
from bs4 import BeautifulSoup
import json
import statistics

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [19]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json" + \
        "?collapse=daily&start_date=2020-11-19&end_date=2020-11-21" + \
        "&api_key=" + API_KEY
url

'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?collapse=daily&start_date=2020-11-19&end_date=2020-11-21&api_key=FsdycwexurRKxfsMkmuk'

In [22]:
r = requests.get(url)

In [23]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2020-11-19',
  'end_date': '2020-11-21',
  'frequency': 'daily',
  'data': [['2020-11-20',
    116.5,
    118.0,
    116.5,
    117.5,
    None,
    295.0,
    34736.0,
    None,
    None,
    None],
   ['2020-11-19',
    115.6,
    115.6,
    115.6,
    115.6,
    None,
    0.0,
    0.0,
    None,
    None,
    None]],
  'collapse': 'daily',
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [24]:
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json" + \
        "?collapse=daily&start_date=2017-1-1&end_date=2017-12-31" + \
        "&api_key=" + API_KEY
url

'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?collapse=daily&start_date=2017-1-1&end_date=2017-12-31&api_key=FsdycwexurRKxfsMkmuk'

In [41]:
#1 Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
r = requests.get(url)
#2 Convert the returned JSON object into a Python dictionary.
data_json = r.json()
data_json_raw = data_json['dataset_data']
type(data_json_raw)

dict

In [42]:
print(data_json_raw.keys())

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [48]:
print(data_json_raw['column_names'])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


In [63]:
AFX_X_Date=[]
AFX_X_Open=[]
AFX_X_Clsoe=[]
AFX_X_Change=[]
AFX_X_Traded_Volume=[]

for entry in data_json_raw['data']:
    of_interest=[entry[0],entry[1],entry[4],entry[5],entry[6]]
    to_be_write=[]
    for item in of_interest[1:]:
        if item != None:
            to_be_write.append(float(item))
        else:
            to_be_write.append(float(0))
    
    AFX_X_Date.append(of_interest[0])
    AFX_X_Open.append(to_be_write[0])
    AFX_X_Clsoe.append(to_be_write[1])
    AFX_X_Change.append(to_be_write[2])
    AFX_X_Traded_Volume.append(to_be_write[3])
    

In [64]:
AFX_X_Open[:5]

[51.76, 51.65, 51.45, 51.05, 51.16]

In [65]:
#3 Calculate what the highest and lowest opening prices were for the stock in this period.
high_op = max(AFX_X_Open)
high_op_loc = AFX_X_Open.index(high_op)
low_op = min(i for i in AFX_X_Open if i > 0)
low_op_loc = AFX_X_Open.index(low_op)
print("High: " + str(high_op) + " " + AFX_X_Date[high_op_loc])
print("Low: " + str(low_op) + " " + AFX_X_Date[low_op_loc])

High: 53.11 2017-12-14
Low: 34.0 2017-01-24


In [75]:
#4 What was the largest change in any one day (based on High and Low price)?
AFX_X_diff=[]
for i in range(len(AFX_X_Open)):
    if AFX_X_Open[i] > 0.0 and AFX_X_Clsoe[i] > 0.0:
        AFX_X_diff.append(abs(AFX_X_Open[i] - AFX_X_Clsoe[i]))
    else:
        AFX_X_diff.append(0)
        
high_diff = max(AFX_X_diff)
high_diff_loc = AFX_X_diff.index(high_diff)
print("Largest diff: " + str(high_diff) + " " + AFX_X_Date[high_diff_loc])

Largest diff: 1.6899999999999977 2017-08-09


In [77]:
#5 What was the largest change between any two days (based on Closing Price)?
AFX_X_diff_inter_day=[]
for i in range(len(AFX_X_Open)-1):
    if AFX_X_Clsoe[i] > 0.0 and AFX_X_Clsoe[i+1] > 0.0:
        AFX_X_diff_inter_day.append(abs(AFX_X_Clsoe[i] - AFX_X_Clsoe[i+1]))
    else:
        AFX_X_diff_inter_day.append(0)
        
high_inter_day = max(AFX_X_diff_inter_day)
high_inter_day_loc = AFX_X_diff_inter_day.index(high_inter_day)
print("Largest diff inter day: " + str(high_inter_day) + " " + AFX_X_Date[high_inter_day_loc])

Largest diff inter day: 2.559999999999995 2017-08-09


In [83]:
#6 What was the average daily trading volume during this year?
AFX_X_Traded_Volume_no_nan=[]
for i in range(len(AFX_X_Traded_Volume)):
    if AFX_X_Traded_Volume[i] > 0.0 :
        AFX_X_Traded_Volume_no_nan.append(AFX_X_Traded_Volume[i])
        
AFX_X_Traded_Volume_no_nan_mean = statistics.mean(AFX_X_Traded_Volume_no_nan)
print("Average daily volume: " + str(AFX_X_Traded_Volume_no_nan_mean))

Average daily volume: 89124.33725490196


In [94]:
#7 (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)
AFX_X_Traded_Volume_no_nan.sort()
median = int(round(len(AFX_X_Traded_Volume_no_nan)/2,0))
print("Median daily volume +1 : " + str(AFX_X_Traded_Volume_no_nan[median-1]))
print("Median daily volume -1 : " + str(AFX_X_Traded_Volume_no_nan[median+1]))
print("True Median daily volume: " + str(AFX_X_Traded_Volume_no_nan[median]))

Median daily volume +1 : 76286.0
Median daily volume -1 : 77420.0
True Median daily volume: 76600.0
